In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["PMIX_MCA_gds"]="hash"

# Import useful packages
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

# To create plots after geting solution data.
import matplotlib.pyplot as plt
import numpy as np

# Packages for the simple design
from SQDMetal.Comps.Junctions import JunctionDolan
import shapely
from SQDMetal.Comps.Polygons import PolyShapely, PolyRectangle
from SQDMetal.Comps.Joints import Joint


# Set up chip design as planar, multiplanar also available
design = designs.DesignPlanar({}, overwrite_enabled=True)

# Set up chip dimensions 
design.chips.main.size.size_x = '800um'
design.chips.main.size.size_y = '800um'
design.chips.main.size.size_z = '500um'
design.chips.main.size.center_x = '0mm'
design.chips.main.size.center_y = '0mm'

JunctionDolan(design, 'junction', options=Dict(pos_x=0, pos_y='-12um', end_x=0, end_y='12um',
                                                         layer=2,
                                                         finger_width='0.4um', t_pad_size='0.385um',
                                                         squid_width='5.4um', prong_width='0.9um'));

#Units in mm...
padCoordNums = [0.19971691, 0.02, 0.17926553, 0.25, 0.25]
padCoords = [[-0.05, 0.012], [0.05, 0.012], [padCoordNums[0], padCoordNums[1]], [padCoordNums[2], padCoordNums[3]], [0, padCoordNums[4]], [-padCoordNums[2], padCoordNums[3]], [-padCoordNums[0], padCoordNums[1]]]
padCoords2 = [[x[0],-x[1]] for x in padCoords][::-1]

# Smooth edges of pads
poly1 = shapely.Polygon(padCoords).buffer(-0.04, join_style=1, quad_segs=4).buffer(0.04, join_style=1, quad_segs=4)
poly2 = shapely.Polygon(padCoords2).buffer(-0.04, join_style=1, quad_segs=4).buffer(0.04, join_style=1, quad_segs=4)

PolyShapely(design, 'pad1', options=dict(strShapely=poly1.__str__()))
PolyShapely(design, 'pad2', options=dict(strShapely=poly2.__str__()))

PolyRectangle(design, 'rectGnd', options=dict(pos_x='-300um',pos_y='-300um', end_x='300um',end_y='300um', is_ground_cutout=True))

Joint(design, 'j1', options=dict(pos_x='0um', pos_y='-300um'))
Joint(design, 'j2', options=dict(pos_x='0um', pos_y='300um'));

# rebuild the GUI
gui = MetalGUI(design)
gui.rebuild()

In [2]:
from SQDMetal.COMSOL.Model import COMSOL_Model
from SQDMetal.COMSOL.SimCapacitance import COMSOL_Simulation_CapMats
from SQDMetal.COMSOL.SimRFsParameter import COMSOL_Simulation_RFsParameters

from SQDMetal.Utilities.ShapelyEx import ShapelyEx


#Initialise the COMSOL engine (needs to only be run once)
COMSOL_Model.init_engine()

#Instantiate a COMSOL model
cmsl = COMSOL_Model('leModel')

#Create simulations to setup - in this case capacitance matrix and RF s-parameter
sim_sParams = COMSOL_Simulation_RFsParameters(cmsl, adaptive='None')

#(A) - Initialise model from Qiskit-Metal design object: design
cmsl.initialize_model(design, [sim_sParams], bottom_grounded=True)

cmsl.add_metallic(1, threshold=1e-20, fuse_threshold=1e-20)
cmsl.add_ground_plane()
cmsl.fuse_all_metals()

sim_sParams.create_port_JosephsonJunction('junction', L_J=4.3e-9, C_J=10e-15)

# sim_sParams.add_surface_current_source_region("dielectric", 0.5)
# sim_sParams.add_surface_current_source_region("metals", 10e-6, 2)

edp_pts = ShapelyEx.get_points_uniform_in_polygon(poly1, 0.02,0.02)
for cur_pt in edp_pts:
    x, y = cur_pt[0]*0.001, cur_pt[1]*0.001 #Converting from mm to m
    sim_sParams.add_electric_point_dipole([x,y, 1e-6], 1, [0,0,1])

cmsl.build_geom_mater_elec_mesh(skip_meshing=True, mesh_structure='Normal')

sim_sParams.set_freq_values([8.118e9])
# cmsl.plot()


# cmsl.save('TestPts')

In [3]:
cmsl.save('TestPts')

In [4]:
sim_sParams.run()

array([[8.118e+09]])

In [5]:
field_data = sim_sParams.eval_fields_over_mesh()

In [6]:
sim_sParams.eval_field_at_pts('E', np.array([[0,0,0], [19e-9,10e-6,0], [0,-10e-6,0], [0,20e-6,0]]))

array([[-7.09522629e+02+3.47338864e+10j,  7.83636538e+06-4.08451998e+14j,
        -6.33434753e+04+7.20785004e+13j],
       [ 1.01766409e+03-4.90214999e+10j,  8.29891057e+06-4.33858885e+14j,
        -5.25061815e+06+3.98586874e+14j],
       [ 1.08888980e+03-5.53311736e+10j,  8.29954292e+06-4.34022836e+14j,
         5.53279776e+06-2.43431765e+14j],
       [-4.41094633e-09-7.28364764e-01j,  1.08284154e-08-5.95161352e+00j,
        -4.16020973e+06+7.11814288e+14j]])

In [7]:
import matplotlib.pyplot as plt

x,y,z,Ez = field_data['coords'][:,0], field_data['coords'][:,1], field_data['coords'][:,2], field_data['E'][:,2]

plane_inds = (np.abs(z)<1e-6)
plt.scatter(x[plane_inds], y[plane_inds], c=np.clip(np.abs(Ez[plane_inds]), 0,1e14))

  offsets = np.asanyarray(offsets, float)



In [8]:
xx,yy = np.meshgrid(np.linspace(-400e-6,400e-6,100), np.linspace(-400e-6,400e-6,100))
xx = np.ndarray.flatten(xx)
yy = np.ndarray.flatten(yy)
zz = np.zeros(xx.size)
coord_xyz = np.vstack([xx,yy,zz]).T

In [13]:
import matplotlib.pyplot as plt

Ezinterpol = sim_sParams.eval_field_at_pts('Ez', coord_xyz)

plane_inds = (np.abs(z)<1e-6)
plt.scatter(xx, yy, c=np.clip(np.abs(Ezinterpol), 0,1e15))
plt.show()

In [14]:
Ezinterpol.shape

(10000,)

In [ ]:
cmsl.save('TestPts')